In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

%matplotlib inline

## Dataset

In [2]:
%%capture
fashion_dataset = keras.datasets.cifar10

(train_images, train_labels), (test_images, test_labels) = fashion_dataset.load_data()

In [3]:
print(train_images.shape)
print(train_labels.shape)

(50000, 32, 32, 3)
(50000, 1)


In [4]:
print("BEFORE NORMALIZING:")
print("IMAGES: ",train_images.max()) # RANGE: 0-255
print("LABELS: ",train_labels.max()) # RANGE: 0-9

BEFORE NORMALIZING:
IMAGES:  255
LABELS:  9


In [5]:
train_images = train_images / 255.0
test_images = test_images / 255.0

print("AFTER NORMALIZING:")
print("IMAGES: ",train_images.max()) # RANGE: 0-1
print("IMAGES: ",test_images.max()) # RANGE: 0-1

AFTER NORMALIZING:
IMAGES:  1.0
IMAGES:  1.0


In [6]:
# plt.figure(figsize=(2,2))
# plt.imshow(train_images[1])

## Image Augmentation

In [7]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# EXPLORE MORE
'''
tf.image.resize      -   resize
image.img_to_array   -   IMG to NP_ARRAY
ImageDataGenerator   -   create augmented dataset
'''

## Network & Train

In [8]:
# NETWORK
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax'),
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), # from_logits = False (DEFAULT) ## EXPLORE ##
              metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)              

In [10]:
model.fit(train_images, train_labels, epochs=10, batch_size=64)

Epoch 1/10
782/782 [==============================] - 60s 77ms/step - loss: 1.5573 - accuracy: 0.4328
Epoch 2/10
782/782 [==============================] - 54s 69ms/step - loss: 1.1984 - accuracy: 0.5738
Epoch 3/10
782/782 [==============================] - 52s 66ms/step - loss: 1.0482 - accuracy: 0.6314
Epoch 4/10
782/782 [==============================] - 55s 70ms/step - loss: 0.9371 - accuracy: 0.6727
Epoch 5/10
782/782 [==============================] - 57s 72ms/step - loss: 0.8562 - accuracy: 0.6987
Epoch 6/10
782/782 [==============================] - 51s 65ms/step - loss: 0.7872 - accuracy: 0.7239
Epoch 7/10
782/782 [==============================] - 54s 69ms/step - loss: 0.7305 - accuracy: 0.7438
Epoch 8/10
782/782 [==============================] - 49s 63ms/step - loss: 0.6851 - accuracy: 0.7600
Epoch 9/10
782/782 [==============================] - 50s 64ms/step - loss: 0.6307 - accuracy: 0.7791
Epoch 10/10
782/782 [==============================] - 50s 64ms/step - loss: 0.594

## Testing

In [11]:
# EVALUATE
test_loss, test_accuracy = model.evaluate(test_images, test_labels)

print("ACC: ",test_accuracy)

313/313 [==============================] - 2s 8ms/step - loss: 0.8699 - accuracy: 0.7042
ACC:  0.704200029373169


In [12]:
# PREDICT
predictions = model.predict(test_images)

print(test_images.shape)
print(predictions.shape)

(10000, 32, 32, 3)
(10000, 10)


## Pretrained Model

In [14]:
# BASE ARCHITECTURE
base_model = tf.keras.applications.MobileNetV2(input_shape=(32,32,3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False # FREEZE WEIGHTS (NON-TRAINABLE)
# base_model.summary()       # ARCHITECTURE

In [15]:
# CUSTOM HEAD
custom_head = keras.Sequential([
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax'),
])

In [16]:
# COMBINED MODEL
model = keras.Sequential([
    base_model,
    custom_head
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy', # from_logits = False (DEFAULT) ## EXPLORE ##
              metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 1, 1, 1280)        2257984   
_________________________________________________________________
sequential_9 (Sequential)    (None, 10)                165258    
Total params: 2,423,242
Trainable params: 165,258
Non-trainable params: 2,257,984
_________________________________________________________________


In [18]:
# FINE-TUNE LAST LAYERS
model.fit(train_images, train_labels, epochs=3, batch_size=64)

Epoch 1/3
782/782 [==============================] - 54s 69ms/step - loss: 2.1996 - accuracy: 0.2327
Epoch 2/3
782/782 [==============================] - 61s 78ms/step - loss: 2.0366 - accuracy: 0.2853
Epoch 3/3
782/782 [==============================] - 63s 80ms/step - loss: 1.9639 - accuracy: 0.2981


In [19]:
# EVALUATE
test_loss, test_accuracy = model.evaluate(test_images, test_labels)

print("ACC: ",test_accuracy)

313/313 [==============================] - 11s 34ms/step - loss: 1.9496 - accuracy: 0.2919
ACC:  0.29190000891685486
